# Word2Vec - Treinamento de word embbeding

### Bibliotecas básicas e outras inicializações

In [1]:
import pandas as pd
import numpy as np
import spacy
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
nlp = spacy.load('pt_core_news_sm')

### Carregando os dados

In [3]:
dados_treino = pd.read_csv('./datasets/treino.csv')
print(dados_treino.shape)
dados_treino.sample()

(90000, 6)


,title,text,date,category,subcategory,link
33679,"Prestes a estrear série, Lázaro Ramos se prepa...","Ao atender à ligação de Jorge Furtado, há algu...",2015-09-13,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/09...


In [4]:
dados_teste = pd.read_csv('./datasets/teste.csv')
print(dados_teste.shape)
dados_teste.sample()

(20513, 6)


,title,text,date,category,subcategory,link
6137,Bruno Soares garante vaga nas quartas de dupla...,O tenista brasileiro Bruno Soares está nas qua...,2015-06-07,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/07/1...


### Pré-processamento do dados

In [5]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
    
    if len(tokens_validos) > 2:
        return ' '.join(tokens_validos)

In [6]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron e le pen turno frança revés siglas trad...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço e alckmin anuncia que...
4,queda maio a atividade econômica sobe junho bc


### Treinamento Word2Vec

#### CBOW

In [7]:
from gensim.models import Word2Vec
w2v_modelo = Word2Vec(sg=0, window=2, size=300, min_count=5, alpha=0.03, min_alpha=0.007)
tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]
w2v_modelo.build_vocab(tokens)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss após a época {}: {}'.format(self.epoch, loss))
        else:
            print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss


w2v_modelo.train(tokens, total_examples=w2v_modelo.corpus_count, epochs=30, compute_loss=True, callbacks=[callback()]) 

Loss após a época 0: 620431.1875
Loss após a época 1: 480012.8125
Loss após a época 2: 449128.375
Loss após a época 3: 415414.5
Loss após a época 4: 362346.625
Loss após a época 5: 335055.5
Loss após a época 6: 316724.25
Loss após a época 7: 299389.75
Loss após a época 8: 283879.25
Loss após a época 9: 267534.0
Loss após a época 10: 255700.0
Loss após a época 11: 228554.75
Loss após a época 12: 210435.5
Loss após a época 13: 211631.0
Loss após a época 14: 193749.5
Loss após a época 15: 178115.5
Loss após a época 16: 179651.0
Loss após a época 17: 174133.0
Loss após a época 18: 159310.0
Loss após a época 19: 161251.5
Loss após a época 20: 158471.0
Loss após a época 21: 159798.5
Loss após a época 22: 149005.0
Loss após a época 23: 145559.0
Loss após a época 24: 140659.0
Loss após a época 25: 137757.5
Loss após a época 26: 127541.0
Loss após a época 27: 131672.5
Loss após a época 28: 134854.0
Loss após a época 29: 126752.0


(14995050, 17644710)

In [9]:
w2v_modelo.wv.most_similar('google')

[('facebook', 0.6091756820678711),
 ('apple', 0.5727623105049133),
 ('amazon', 0.5125503540039062),
 ('uber', 0.4780091643333435),
 ('yahoo', 0.4703161120414734),
 ('tesla', 0.4588194489479065),
 ('airbnb', 0.4499620497226715),
 ('walmart', 0.4396229386329651),
 ('waze', 0.43843623995780945),
 ('alibaba', 0.43683409690856934)]

#### SKIP-GRAM

In [10]:
w2v_modelo_sg = Word2Vec(sg=1, window=5, size=300, min_count=5, alpha=0.03, min_alpha=0.007)
w2v_modelo_sg.build_vocab(tokens)
w2v_modelo_sg.train(tokens, total_examples=w2v_modelo.corpus_count, epochs=30, compute_loss=True, callbacks=[callback()]) 

Loss após a época 0: 1924311.875
Loss após a época 1: 1520951.625
Loss após a época 2: 1307483.0
Loss após a época 3: 1155441.5
Loss após a época 4: 1174818.0
Loss após a época 5: 1088112.5
Loss após a época 6: 949190.5
Loss após a época 7: 902036.0
Loss após a época 8: 878130.0
Loss após a época 9: 863686.0
Loss após a época 10: 797922.0
Loss após a época 11: 783432.0
Loss após a época 12: 766337.0
Loss após a época 13: 786010.0
Loss após a época 14: 766761.0
Loss após a época 15: 748570.0
Loss após a época 16: 645185.0
Loss após a época 17: 550600.0
Loss após a época 18: 545014.0
Loss após a época 19: 527192.0
Loss após a época 20: 516716.0
Loss após a época 21: 506732.0
Loss após a época 22: 498592.0
Loss após a época 23: 488752.0
Loss após a época 24: 481176.0
Loss após a época 25: 453590.0
Loss após a época 26: 469396.0
Loss após a época 27: 461542.0
Loss após a época 28: 454000.0
Loss após a época 29: 446070.0


(14995519, 17644710)

In [11]:
w2v_modelo_sg.wv.most_similar('google')

[('reguladores', 0.45003873109817505),
 ('android', 0.41569453477859497),
 ('buffett', 0.41071152687072754),
 ('patentes', 0.4014918804168701),
 ('apple', 0.3949420154094696),
 ('concorda', 0.3683273196220398),
 ('yahoo', 0.3673931658267975),
 ('waze', 0.3651007413864136),
 ('warren', 0.35934484004974365),
 ('toshiba', 0.357677698135376)]